### Extracting and Renaming

In [4]:
import re
import tarfile
from pathlib import Path
import pandas as pd
import shutil
import os

In [5]:
FPS = 15
# directory= '14-3-2024_#15_INDIVIDUAL_(18)' #10 chunks
directory= '8-5-2024_#18_INDIVIDUAL_[12]'     #7 chunks  
BASE_PATH = f"/home/liubov/Bureau/new/{directory}/PosesDir"
base_path = Path(BASE_PATH)

# Find all chunk folders
chunk_dirs = sorted(base_path.glob("chunk_*"))
print(f"Found {len(chunk_dirs)} chunk folders:")
for cd in chunk_dirs:
    print(f"  - {cd.name}")

Found 7 chunk folders:
  - chunk_00_00000-00120s
  - chunk_01_00120-00240s
  - chunk_02_00240-00360s
  - chunk_03_00360-00480s
  - chunk_04_00480-00600s
  - chunk_05_00600-00720s
  - chunk_06_00720-00785s


In [6]:
total_renamed = 0

for chunk_dir in chunk_dirs:
    # Parse start/end seconds from folder name
    m = re.search(r"chunk_\d+_(\d+)-(\d+)s$", chunk_dir.name)
    if not m:
        print(f"[SKIP] Can't parse chunk range from: {chunk_dir}")
        continue
    start_sec, end_sec = map(int, m.groups())
    abs_base = start_sec * FPS

    print(f"\n=== Processing {chunk_dir} ({start_sec:05}-{end_sec:05}s) ===")

    # Find all tar.gz files in this chunk folder
    tar_files = sorted(chunk_dir.glob("*.tar.gz"))
    if not tar_files:
        print("  No tar.gz files found.")
        continue

    for tar_file in tar_files:
        print(f"  Extracting {tar_file.name}...")

        extract_dir = tar_file.with_name(f"{tar_file.stem}_extracted")
        extract_dir.mkdir(exist_ok=True)

        try:
            with tarfile.open(tar_file, "r:gz") as tar:
                for member in tar.getmembers():
                    member.path = Path(member.name).name  # flatten
                    tar.extract(member, path=extract_dir)
        except Exception as e:
            print(f"  Error extracting {tar_file.name}: {e}")
            continue

        # Rename JSON files
        json_files = list(extract_dir.rglob("*.json"))
        if not json_files:
            print("   No JSON files found.")
            continue

        # video_id can be derived from tar file name or fixed
        video_id = tar_file.stem.split('_')[0]

        for jf in json_files:
            stem = jf.stem
            mnum = re.search(r"(\d+)$", stem)
            if not mnum:
                print(f"    Skipping (no trailing number): {jf.name}")
                continue

            local_idx = int(mnum.group(1))
            abs_idx = abs_base + local_idx

            new_name = f"id_{video_id}_{start_sec:05}-{end_sec:05}s_{abs_idx:04}.json"
            new_path = jf.with_name(new_name)

            if jf.name == new_name:
                continue

            if new_path.exists():
                print(f"    Warning: target exists, skipping: {new_name}")
                continue

            jf.rename(new_path)
            print(f"    {jf.name} -> {new_name}")
            total_renamed += 1


=== Processing /home/liubov/Bureau/new/8-5-2024_#18_INDIVIDUAL_[12]/PosesDir/chunk_00_00000-00120s (00000-00120s) ===
  Extracting 1.tar.gz...
     00000000000675.json -> id_1.tar_00000-00120s_0675.json
     00000000001175.json -> id_1.tar_00000-00120s_1175.json
     00000000001170.json -> id_1.tar_00000-00120s_1170.json
     00000000001363.json -> id_1.tar_00000-00120s_1363.json
     00000000001349.json -> id_1.tar_00000-00120s_1349.json
     00000000001204.json -> id_1.tar_00000-00120s_1204.json
     00000000000510.json -> id_1.tar_00000-00120s_0510.json
     00000000000593.json -> id_1.tar_00000-00120s_0593.json
     00000000001774.json -> id_1.tar_00000-00120s_1774.json
     00000000001420.json -> id_1.tar_00000-00120s_1420.json
     00000000000775.json -> id_1.tar_00000-00120s_0775.json
     00000000000095.json -> id_1.tar_00000-00120s_0095.json
     00000000000433.json -> id_1.tar_00000-00120s_0433.json
     00000000001470.json -> id_1.tar_00000-00120s_1470.json
     00000000000

     00000000000675.json -> id_2.tar_00000-00120s_0675.json
     00000000001175.json -> id_2.tar_00000-00120s_1175.json
     00000000001170.json -> id_2.tar_00000-00120s_1170.json
     00000000001363.json -> id_2.tar_00000-00120s_1363.json
     00000000001349.json -> id_2.tar_00000-00120s_1349.json
     00000000001204.json -> id_2.tar_00000-00120s_1204.json
     00000000000510.json -> id_2.tar_00000-00120s_0510.json
     00000000000593.json -> id_2.tar_00000-00120s_0593.json
     00000000001774.json -> id_2.tar_00000-00120s_1774.json
     00000000001420.json -> id_2.tar_00000-00120s_1420.json
     00000000000775.json -> id_2.tar_00000-00120s_0775.json
     00000000000095.json -> id_2.tar_00000-00120s_0095.json
     00000000000433.json -> id_2.tar_00000-00120s_0433.json
     00000000001470.json -> id_2.tar_00000-00120s_1470.json
     00000000000794.json -> id_2.tar_00000-00120s_0794.json
     00000000000756.json -> id_2.tar_00000-00120s_0756.json
     00000000001235.json -> id_2.tar_000

     00000000000976.json -> id_2.tar_00000-00120s_0976.json
     00000000000006.json -> id_2.tar_00000-00120s_0006.json
     00000000000930.json -> id_2.tar_00000-00120s_0930.json
     00000000001157.json -> id_2.tar_00000-00120s_1157.json
     00000000001723.json -> id_2.tar_00000-00120s_1723.json
     00000000000803.json -> id_2.tar_00000-00120s_0803.json
     00000000000919.json -> id_2.tar_00000-00120s_0919.json
     00000000001161.json -> id_2.tar_00000-00120s_1161.json
     00000000000298.json -> id_2.tar_00000-00120s_0298.json
     00000000001756.json -> id_2.tar_00000-00120s_1756.json
     00000000001425.json -> id_2.tar_00000-00120s_1425.json
     00000000000216.json -> id_2.tar_00000-00120s_0216.json
     00000000001277.json -> id_2.tar_00000-00120s_1277.json
     00000000000366.json -> id_2.tar_00000-00120s_0366.json
     00000000000656.json -> id_2.tar_00000-00120s_0656.json
     00000000001373.json -> id_2.tar_00000-00120s_1373.json
     00000000001432.json -> id_2.tar_000

     00000000000675.json -> id_3.tar_00000-00120s_0675.json
     00000000001175.json -> id_3.tar_00000-00120s_1175.json
     00000000001170.json -> id_3.tar_00000-00120s_1170.json
     00000000001363.json -> id_3.tar_00000-00120s_1363.json
     00000000001349.json -> id_3.tar_00000-00120s_1349.json
     00000000001204.json -> id_3.tar_00000-00120s_1204.json
     00000000000510.json -> id_3.tar_00000-00120s_0510.json
     00000000000593.json -> id_3.tar_00000-00120s_0593.json
     00000000001774.json -> id_3.tar_00000-00120s_1774.json
     00000000001420.json -> id_3.tar_00000-00120s_1420.json
     00000000000775.json -> id_3.tar_00000-00120s_0775.json
     00000000000095.json -> id_3.tar_00000-00120s_0095.json
     00000000000433.json -> id_3.tar_00000-00120s_0433.json
     00000000001470.json -> id_3.tar_00000-00120s_1470.json
     00000000000794.json -> id_3.tar_00000-00120s_0794.json
     00000000000756.json -> id_3.tar_00000-00120s_0756.json
     00000000001235.json -> id_3.tar_000

     00000000000476.json -> id_3.tar_00000-00120s_0476.json
     00000000001227.json -> id_3.tar_00000-00120s_1227.json
     00000000001543.json -> id_3.tar_00000-00120s_1543.json
     00000000001289.json -> id_3.tar_00000-00120s_1289.json
     00000000001212.json -> id_3.tar_00000-00120s_1212.json
     00000000001794.json -> id_3.tar_00000-00120s_1794.json
     00000000000179.json -> id_3.tar_00000-00120s_0179.json
     00000000001144.json -> id_3.tar_00000-00120s_1144.json
     00000000001783.json -> id_3.tar_00000-00120s_1783.json
     00000000000799.json -> id_3.tar_00000-00120s_0799.json
     00000000001724.json -> id_3.tar_00000-00120s_1724.json
     00000000000348.json -> id_3.tar_00000-00120s_0348.json
     00000000000202.json -> id_3.tar_00000-00120s_0202.json
     00000000001680.json -> id_3.tar_00000-00120s_1680.json
     00000000001550.json -> id_3.tar_00000-00120s_1550.json
     00000000001766.json -> id_3.tar_00000-00120s_1766.json
     00000000000276.json -> id_3.tar_000

...

     00000000000675.json -> id_3.tar_00600-00720s_9675.json
     00000000001175.json -> id_3.tar_00600-00720s_10175.json
     00000000001170.json -> id_3.tar_00600-00720s_10170.json
     00000000001363.json -> id_3.tar_00600-00720s_10363.json
     00000000001349.json -> id_3.tar_00600-00720s_10349.json
     00000000001204.json -> id_3.tar_00600-00720s_10204.json
     00000000000510.json -> id_3.tar_00600-00720s_9510.json
     00000000000593.json -> id_3.tar_00600-00720s_9593.json
     00000000001774.json -> id_3.tar_00600-00720s_10774.json
     00000000001420.json -> id_3.tar_00600-00720s_10420.json
     00000000000775.json -> id_3.tar_00600-00720s_9775.json
     00000000000095.json -> id_3.tar_00600-00720s_9095.json
     00000000000433.json -> id_3.tar_00600-00720s_9433.json
     00000000001470.json -> id_3.tar_00600-00720s_10470.json
     00000000000794.json -> id_3.tar_00600-00720s_9794.json
     00000000000756.json -> id_3.tar_00600-00720s_9756.json
     00000000001235.json -> id_3

     00000000001028.json -> id_3.tar_00600-00720s_10028.json
     00000000001719.json -> id_3.tar_00600-00720s_10719.json
     00000000001741.json -> id_3.tar_00600-00720s_10741.json
     00000000001089.json -> id_3.tar_00600-00720s_10089.json
     00000000000339.json -> id_3.tar_00600-00720s_9339.json
     00000000000733.json -> id_3.tar_00600-00720s_9733.json
     00000000000561.json -> id_3.tar_00600-00720s_9561.json
     00000000000828.json -> id_3.tar_00600-00720s_9828.json
     00000000001617.json -> id_3.tar_00600-00720s_10617.json
     00000000000541.json -> id_3.tar_00600-00720s_9541.json
     00000000001571.json -> id_3.tar_00600-00720s_10571.json
     00000000001064.json -> id_3.tar_00600-00720s_10064.json
     00000000001748.json -> id_3.tar_00600-00720s_10748.json
     00000000001352.json -> id_3.tar_00600-00720s_10352.json
     00000000000492.json -> id_3.tar_00600-00720s_9492.json
     00000000001203.json -> id_3.tar_00600-00720s_10203.json
     00000000001041.json -> id

     00000000000208.json -> id_1.tar_00720-00785s_11008.json
     00000000000472.json -> id_1.tar_00720-00785s_11272.json
     00000000000664.json -> id_1.tar_00720-00785s_11464.json
     00000000000595.json -> id_1.tar_00720-00785s_11395.json
     00000000000327.json -> id_1.tar_00720-00785s_11127.json
     00000000000657.json -> id_1.tar_00720-00785s_11457.json
     00000000000086.json -> id_1.tar_00720-00785s_10886.json
     00000000000872.json -> id_1.tar_00720-00785s_11672.json
     00000000000262.json -> id_1.tar_00720-00785s_11062.json
     00000000000371.json -> id_1.tar_00720-00785s_11171.json
     00000000000786.json -> id_1.tar_00720-00785s_11586.json
     00000000000529.json -> id_1.tar_00720-00785s_11329.json
     00000000000486.json -> id_1.tar_00720-00785s_11286.json
     00000000000133.json -> id_1.tar_00720-00785s_10933.json
     00000000000946.json -> id_1.tar_00720-00785s_11746.json
     00000000000176.json -> id_1.tar_00720-00785s_10976.json
     00000000000701.json

     00000000000372.json -> id_2.tar_00720-00785s_11172.json
     00000000000983.json -> id_2.tar_00720-00785s_11783.json
     00000000000478.json -> id_2.tar_00720-00785s_11278.json
     00000000000581.json -> id_2.tar_00720-00785s_11381.json
     00000000000948.json -> id_2.tar_00720-00785s_11748.json
     00000000000480.json -> id_2.tar_00720-00785s_11280.json
     00000000000698.json -> id_2.tar_00720-00785s_11498.json
     00000000000048.json -> id_2.tar_00720-00785s_10848.json
     00000000000365.json -> id_2.tar_00720-00785s_11165.json
     00000000000259.json -> id_2.tar_00720-00785s_11059.json
     00000000000227.json -> id_2.tar_00720-00785s_11027.json
     00000000000771.json -> id_2.tar_00720-00785s_11571.json
     00000000000508.json -> id_2.tar_00720-00785s_11308.json
     00000000000602.json -> id_2.tar_00720-00785s_11402.json
     00000000000246.json -> id_2.tar_00720-00785s_11046.json
     00000000000500.json -> id_2.tar_00720-00785s_11300.json
     00000000000443.json

     00000000000235.json -> id_3.tar_00720-00785s_11035.json
     00000000000504.json -> id_3.tar_00720-00785s_11304.json
     00000000000497.json -> id_3.tar_00720-00785s_11297.json
     00000000000438.json -> id_3.tar_00720-00785s_11238.json
     00000000000977.json -> id_3.tar_00720-00785s_11777.json
     00000000000253.json -> id_3.tar_00720-00785s_11053.json
     00000000000729.json -> id_3.tar_00720-00785s_11529.json
     00000000000981.json -> id_3.tar_00720-00785s_11781.json
     00000000000770.json -> id_3.tar_00720-00785s_11570.json
     00000000000538.json -> id_3.tar_00720-00785s_11338.json
     00000000000622.json -> id_3.tar_00720-00785s_11422.json
     00000000000758.json -> id_3.tar_00720-00785s_11558.json
     00000000000562.json -> id_3.tar_00720-00785s_11362.json
     00000000000567.json -> id_3.tar_00720-00785s_11367.json
     00000000000307.json -> id_3.tar_00720-00785s_11107.json
     00000000000890.json -> id_3.tar_00720-00785s_11690.json
     00000000000055.json

In [7]:
print(f"\n Summary ")
print(f"Total JSON files renamed: {total_renamed}")


 Summary 
Total JSON files renamed: 35355


### Collect all json-files

In [8]:
import os
import shutil

# Base directory where all your folders are
base_dir = BASE_PATH

# Destination folder where all JSONs will be copied
dest_dir = os.path.join(base_dir, "all_jsons")
os.makedirs(dest_dir, exist_ok=True)

# Loop through all subdirectories
for root, dirs, files in os.walk(base_dir):
    # Skip the all_jsons directory itself
    if root == dest_dir:
        continue

    for file in files:
        if file.endswith(".json"):
            src_file = os.path.join(root, file)
            dst_file = os.path.join(dest_dir, file)

            # Skip if source == destination
            if os.path.abspath(src_file) == os.path.abspath(dst_file):
                continue

            shutil.copy2(src_file, dst_file)

print("All JSON files have been copied to:", dest_dir)

All JSON files have been copied to: /home/liubov/Bureau/new/8-5-2024_#18_INDIVIDUAL_[12]/PosesDir/all_jsons


In [9]:
import os
import shutil

# Destination folder where all JSONs will be copied
dest_dir = os.path.join(base_dir, "all_jsons")
os.makedirs(dest_dir, exist_ok=True)

# Loop through all subdirectories
for root, dirs, files in os.walk(base_dir):
    # prevent walking into all_jsons
    if os.path.abspath(root) == os.path.abspath(dest_dir):
        continue

    for file in files:
        if file.endswith(".json"):
            src_file = os.path.join(root, file)
            dst_file = os.path.join(dest_dir, file)

            # extra safety: skip if src == dst
            if os.path.abspath(src_file) == os.path.abspath(dst_file):
                continue

            shutil.copy2(src_file, dst_file)

print("All JSON files have been copied to:", dest_dir)

All JSON files have been copied to: /home/liubov/Bureau/new/8-5-2024_#18_INDIVIDUAL_[12]/PosesDir/all_jsons


### Download and Preparing Mapping File

In [10]:
mapping_df=pd.read_csv(f'/home/liubov/Bureau/new/{directory}/first_id_mapping_camera_a.csv')

In [11]:
mapping_df.drop(columns='Unnamed: 0', inplace=True)

In [12]:
mapping_df

,frame_number,original_id,updated_id
0,1800,1,3
1,1800,2,1
2,1800,3,2
3,3600,1,1
4,3600,2,2
5,3600,3,3
6,5400,1,3
7,5400,2,1
8,5400,3,2
9,7200,1,3


In [13]:
all_jsons_dir = f"{base_dir}/all_jsons"

frame_numbers = []
pattern = re.compile(r'_(\d+)\.json$')  # captures the last number before .json

for file in os.listdir(all_jsons_dir):
    if file.endswith(".json"):
        match = pattern.search(file)
        if match:
            frame_numbers.append(int(match.group(1)))

if frame_numbers:
    max_frame = max(frame_numbers)
else:
    raise ValueError("No JSON files with frame numbers found!")

print("Max frame number found:", max_frame)

# Example: your existing mapping_df logic
mapping_df["next_frame_number"] = mapping_df.groupby("frame_number")["frame_number"].transform("max").shift(-3, fill_value=max_frame)

# Expand mapping_df
expanded = mapping_df.apply(
    lambda row: pd.DataFrame({
        "frame_number": range(row["frame_number"], row["next_frame_number"]),
        "original_id": row["original_id"],
        "updated_id": row["updated_id"]
    }), axis=1
)

expanded_df = pd.concat(expanded.tolist(), ignore_index=True)

print(expanded_df)

Max frame number found: 11784
       frame_number  original_id  updated_id
0              1800            1           3
1              1801            1           3
2              1802            1           3
3              1803            1           3
4              1804            1           3
...             ...          ...         ...
29947         11779            3           2
29948         11780            3           2
29949         11781            3           2
29950         11782            3           2
29951         11783            3           2

[29952 rows x 3 columns]


In [11]:
# expanded_df.tail(10)

In [12]:
# mapping_df.tail()

### Original ID to Updated ID Renaming

In [14]:
json_dir = Path(all_jsons_dir)

json_directory_path_new = f'{base_dir}/new_jsons'
output_dir = Path(json_directory_path_new)
output_dir.mkdir(exist_ok=True)

json_files = list(json_dir.glob("*.json"))
print(f"Found {len(json_files)} JSON files")

Found 35356 JSON files


In [15]:
pattern = re.compile(r"^id_(\d+)\.tar_\d+-\d+s_(\d+)\.json$")

renamed_count = 0
copied_count = 0

for json_file in json_files:
    filename = json_file.name
    old_path = json_file

    match = pattern.match(filename)
    if match:
        try:
            original_id = int(match.group(1))
            frame_number = int(match.group(2))

            # Try to find mapping
            matching_rows = expanded_df[
                (expanded_df['frame_number'] == frame_number) &
                (expanded_df['original_id'] == original_id)
            ]

            if len(matching_rows) > 0:
                updated_id = matching_rows.iloc[0]['updated_id']
            else:
                print(f"  No mapping found for ID {original_id}, frame {frame_number} - using original ID")
                updated_id = original_id

            # Always generate new filename in the unified format
            new_filename = f"id_{updated_id}_{frame_number}.json"
            new_path = output_dir / new_filename

            if new_path.exists():
                print(f"  WARNING: Target file {new_filename} already exists!")
                continue

            shutil.copy2(str(old_path), str(new_path))

            if len(matching_rows) > 0:
                renamed_count += 1
            else:
                copied_count += 1

        except Exception as e:
            print(f"  Error processing file {filename}: {e}")
            continue

    else:
        # Doesn’t match pattern → fallback: just copy with same name (or make up format?)
        print(f"Skipping file (pattern mismatch): {filename}")
        new_path = output_dir / filename
        if not new_path.exists():
            shutil.copy2(str(old_path), str(new_path))
            copied_count += 1
        else:
            print(f"  WARNING: Target file {filename} already exists!")

print(f"\nCompleted! Renamed {renamed_count} files, copied {copied_count} files with original IDs.")


  No mapping found for ID 1, frame 449 - using original ID
  No mapping found for ID 1, frame 1637 - using original ID
  No mapping found for ID 2, frame 1671 - using original ID
  No mapping found for ID 3, frame 1508 - using original ID
  No mapping found for ID 3, frame 294 - using original ID
  No mapping found for ID 2, frame 1583 - using original ID
  No mapping found for ID 1, frame 1154 - using original ID
  No mapping found for ID 2, frame 89 - using original ID
  No mapping found for ID 3, frame 1175 - using original ID
  No mapping found for ID 2, frame 1788 - using original ID
  No mapping found for ID 1, frame 1068 - using original ID
  No mapping found for ID 3, frame 984 - using original ID
  No mapping found for ID 2, frame 780 - using original ID
  No mapping found for ID 1, frame 1422 - using original ID
  No mapping found for ID 2, frame 1168 - using original ID
  No mapping found for ID 3, frame 1639 - using original ID
  No mapping found for ID 1, frame 1339 - usin

  No mapping found for ID 1, frame 546 - using original ID
  No mapping found for ID 1, frame 1179 - using original ID
  No mapping found for ID 2, frame 489 - using original ID
  No mapping found for ID 2, frame 0 - using original ID
  No mapping found for ID 3, frame 440 - using original ID
  No mapping found for ID 1, frame 607 - using original ID
  No mapping found for ID 1, frame 1008 - using original ID
  No mapping found for ID 1, frame 662 - using original ID
  No mapping found for ID 1, frame 491 - using original ID
  No mapping found for ID 3, frame 499 - using original ID
  No mapping found for ID 2, frame 536 - using original ID
  No mapping found for ID 2, frame 1312 - using original ID
  No mapping found for ID 1, frame 706 - using original ID
  No mapping found for ID 3, frame 1135 - using original ID
  No mapping found for ID 2, frame 1315 - using original ID
  No mapping found for ID 3, frame 184 - using original ID
  No mapping found for ID 2, frame 924 - using origin

  No mapping found for ID 2, frame 111 - using original ID
  No mapping found for ID 1, frame 8 - using original ID
  No mapping found for ID 1, frame 970 - using original ID
  No mapping found for ID 2, frame 1672 - using original ID
  No mapping found for ID 2, frame 1157 - using original ID
  No mapping found for ID 3, frame 1284 - using original ID
  No mapping found for ID 3, frame 1169 - using original ID
  No mapping found for ID 2, frame 1648 - using original ID
  No mapping found for ID 1, frame 38 - using original ID
  No mapping found for ID 1, frame 213 - using original ID
  No mapping found for ID 3, frame 218 - using original ID
  No mapping found for ID 1, frame 543 - using original ID
  No mapping found for ID 3, frame 1786 - using original ID
  No mapping found for ID 3, frame 1648 - using original ID
  No mapping found for ID 1, frame 344 - using original ID
  No mapping found for ID 2, frame 566 - using original ID
  No mapping found for ID 1, frame 295 - using origi

  No mapping found for ID 2, frame 1573 - using original ID
  No mapping found for ID 2, frame 1262 - using original ID
  No mapping found for ID 1, frame 937 - using original ID
  No mapping found for ID 1, frame 552 - using original ID
  No mapping found for ID 1, frame 1369 - using original ID
  No mapping found for ID 3, frame 1611 - using original ID
  No mapping found for ID 1, frame 925 - using original ID
  No mapping found for ID 2, frame 199 - using original ID
  No mapping found for ID 1, frame 1060 - using original ID
  No mapping found for ID 2, frame 1019 - using original ID
  No mapping found for ID 2, frame 157 - using original ID
  No mapping found for ID 3, frame 898 - using original ID
  No mapping found for ID 2, frame 1640 - using original ID
  No mapping found for ID 1, frame 975 - using original ID
  No mapping found for ID 3, frame 670 - using original ID
  No mapping found for ID 1, frame 1670 - using original ID
  No mapping found for ID 2, frame 1062 - using 

  No mapping found for ID 3, frame 427 - using original ID
  No mapping found for ID 2, frame 67 - using original ID
  No mapping found for ID 2, frame 529 - using original ID
  No mapping found for ID 2, frame 502 - using original ID
  No mapping found for ID 2, frame 1366 - using original ID
  No mapping found for ID 1, frame 1266 - using original ID
  No mapping found for ID 1, frame 1228 - using original ID
  No mapping found for ID 3, frame 401 - using original ID
  No mapping found for ID 1, frame 382 - using original ID
  No mapping found for ID 2, frame 1333 - using original ID
  No mapping found for ID 3, frame 639 - using original ID
  No mapping found for ID 3, frame 1201 - using original ID
  No mapping found for ID 1, frame 532 - using original ID
  No mapping found for ID 2, frame 1010 - using original ID
  No mapping found for ID 1, frame 1080 - using original ID
  No mapping found for ID 3, frame 1245 - using original ID
  No mapping found for ID 2, frame 1 - using orig

...

  No mapping found for ID 3, frame 355 - using original ID
  No mapping found for ID 3, frame 759 - using original ID
  No mapping found for ID 1, frame 943 - using original ID
  No mapping found for ID 2, frame 975 - using original ID
  No mapping found for ID 2, frame 1133 - using original ID
  No mapping found for ID 2, frame 1434 - using original ID
  No mapping found for ID 3, frame 525 - using original ID
  No mapping found for ID 1, frame 1139 - using original ID
  No mapping found for ID 3, frame 961 - using original ID
  No mapping found for ID 2, frame 19 - using original ID
  No mapping found for ID 1, frame 1136 - using original ID
  No mapping found for ID 3, frame 1199 - using original ID
  No mapping found for ID 3, frame 510 - using original ID
  No mapping found for ID 1, frame 1147 - using original ID
  No mapping found for ID 2, frame 1664 - using original ID
  No mapping found for ID 2, frame 644 - using original ID
  No mapping found for ID 2, frame 1413 - using or

  No mapping found for ID 1, frame 1357 - using original ID
  No mapping found for ID 2, frame 1418 - using original ID
  No mapping found for ID 2, frame 725 - using original ID
  No mapping found for ID 2, frame 486 - using original ID
  No mapping found for ID 2, frame 824 - using original ID
  No mapping found for ID 2, frame 1231 - using original ID
  No mapping found for ID 1, frame 1130 - using original ID
  No mapping found for ID 3, frame 1274 - using original ID
  No mapping found for ID 3, frame 974 - using original ID
  No mapping found for ID 2, frame 154 - using original ID
  No mapping found for ID 2, frame 935 - using original ID
  No mapping found for ID 1, frame 1335 - using original ID
  No mapping found for ID 2, frame 736 - using original ID
  No mapping found for ID 1, frame 438 - using original ID
  No mapping found for ID 2, frame 174 - using original ID
  No mapping found for ID 2, frame 1066 - using original ID
  No mapping found for ID 2, frame 1110 - using o

  No mapping found for ID 3, frame 986 - using original ID
  No mapping found for ID 2, frame 1288 - using original ID
  No mapping found for ID 3, frame 1622 - using original ID
  No mapping found for ID 2, frame 1296 - using original ID
  No mapping found for ID 3, frame 182 - using original ID
  No mapping found for ID 1, frame 1645 - using original ID
  No mapping found for ID 1, frame 561 - using original ID
  No mapping found for ID 3, frame 1729 - using original ID
  No mapping found for ID 3, frame 836 - using original ID
  No mapping found for ID 1, frame 441 - using original ID
  No mapping found for ID 1, frame 1142 - using original ID
  No mapping found for ID 1, frame 932 - using original ID
  No mapping found for ID 3, frame 882 - using original ID
  No mapping found for ID 3, frame 1190 - using original ID
  No mapping found for ID 1, frame 214 - using original ID
  No mapping found for ID 1, frame 528 - using original ID
  No mapping found for ID 1, frame 812 - using or

  No mapping found for ID 1, frame 1230 - using original ID
  No mapping found for ID 1, frame 1582 - using original ID
  No mapping found for ID 1, frame 1455 - using original ID
  No mapping found for ID 2, frame 913 - using original ID
  No mapping found for ID 1, frame 515 - using original ID
  No mapping found for ID 1, frame 1468 - using original ID
  No mapping found for ID 1, frame 990 - using original ID
  No mapping found for ID 1, frame 1529 - using original ID
  No mapping found for ID 1, frame 157 - using original ID
  No mapping found for ID 3, frame 997 - using original ID
  No mapping found for ID 1, frame 1688 - using original ID
  No mapping found for ID 3, frame 1770 - using original ID
  No mapping found for ID 2, frame 792 - using original ID
  No mapping found for ID 3, frame 1149 - using original ID
  No mapping found for ID 1, frame 475 - using original ID
  No mapping found for ID 2, frame 287 - using original ID
  No mapping found for ID 1, frame 1375 - using 

  No mapping found for ID 1, frame 834 - using original ID
  No mapping found for ID 2, frame 1109 - using original ID
  No mapping found for ID 3, frame 1126 - using original ID
  No mapping found for ID 1, frame 1180 - using original ID
  No mapping found for ID 2, frame 1643 - using original ID
  No mapping found for ID 3, frame 1039 - using original ID
  No mapping found for ID 1, frame 348 - using original ID
  No mapping found for ID 2, frame 117 - using original ID
  No mapping found for ID 3, frame 1316 - using original ID
  No mapping found for ID 3, frame 651 - using original ID
  No mapping found for ID 1, frame 906 - using original ID
  No mapping found for ID 3, frame 1266 - using original ID
  No mapping found for ID 3, frame 723 - using original ID
  No mapping found for ID 2, frame 1625 - using original ID
  No mapping found for ID 3, frame 52 - using original ID
  No mapping found for ID 3, frame 1035 - using original ID
  No mapping found for ID 2, frame 1108 - using 

### JSONs Merging

In [17]:
import json
pattern = re.compile(r"id_\d+")
grouped_data = {}

for filename in os.listdir(json_directory_path_new):
    if filename.endswith(".json"):
        match = pattern.search(filename)
        if match:
            group = match.group()  # e.g., "id_1"
            grouped_data.setdefault(group, []).append(filename)

for group, files in grouped_data.items():
    files.sort()

    merged_data = []
    for filename in files:
        file_path = os.path.join(json_directory_path_new, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            merged_data.append(data)

    output_file = f"{json_directory_path_new}_merged_{group}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(merged_data, f, indent=4, ensure_ascii=False)

    print(f"Merged {group} ({len(files)} files) -> {output_file}")

Merged id_2 (11785 files) -> /home/liubov/Bureau/new/8-5-2024_#18_INDIVIDUAL_[12]/PosesDir/new_jsons_merged_id_2.json
Merged id_3 (11785 files) -> /home/liubov/Bureau/new/8-5-2024_#18_INDIVIDUAL_[12]/PosesDir/new_jsons_merged_id_3.json
Merged id_1 (11785 files) -> /home/liubov/Bureau/new/8-5-2024_#18_INDIVIDUAL_[12]/PosesDir/new_jsons_merged_id_1.json
